In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [3]:
stories6y = pd.read_csv('6y.csv',index_col=[0] )

In [4]:
# replace non values with nan
stories6y.fillna(value=np.nan, inplace=True)


In [25]:
# filter out delted stories
stories6y_f = stories6y[stories6y.deleted.isnull()]

In [51]:
s6y = stories6y_f[['title','url']]

In [28]:
from gensim import corpora
dictionary = corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load( open( "corpus.pkl", "rb" ) )


In [13]:
import gensim
ldamodel = gensim.models.ldamodel.LdaModel.load('hn15.gensim')

In [44]:
# extract news under the same topic 
post_topic =  np.zeros((len(corpus),2))


In [49]:
for i, row in enumerate(ldamodel[corpus]):
    row = sorted(row, key=lambda x: (x[1]), reverse=True)
    post_topic[i,0] = row[0][0]
    post_topic[i,1] = row[0][1]

In [55]:
pt_df = pd.DataFrame(post_topic, index = s6y.index, columns = ['topic','percentage'])

In [57]:
news_df = pd.concat([s6y, pt_df], axis=1)

In [59]:
news_df.to_csv('news_df.csv')

In [70]:
news_df = pd.read_csv('news_df.csv',index_col=[0] )

D:\python3\envs\tdi\lib\site-packages\numpy\lib\arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [74]:
news_df6 = news_df[news_df.topic == 6.0]

In [75]:
news_df6s =news_df6.sort_values(by =['percentage'],ascending=False)

In [76]:
# set a threshold and use the more certain posts for autoencoder training
news_df6s = news_df6s[news_df6s.percentage >= 0.4]

In [25]:
import tldextract
domains = news_df6s.url.fillna(' ')

In [26]:
maindomain = domains.apply(lambda x: tldextract.extract(x).domain)

In [36]:
# construct the training data
traintitle = news_df6s.title.fillna(' ').astype(str) + ' ' + maindomain

In [3]:
# use glove embeddings
EMBEDDING_FILE = 'glove.840B.300d.txt'

# Embedding file organized as word, followed by the vector
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8"))

In [39]:
# add spaces around punctures to separate from words
def add_spaces(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x
# numbers are represented as #, ##, ..(according to their length) in glove
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x
# clean different forms of 's
def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text
def replace_words(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x
# to be replaced
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
mispell_mapping = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization', 'pokémon': 'pokemon'}
knownpunc_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

In [40]:
# replace words
traintitle = traintitle.apply(lambda x: replace_words(x,contraction_mapping))
traintitle = traintitle.apply(lambda x: replace_words(x,mispell_mapping))
traintitle = traintitle.apply(lambda x: replace_words(x,knownpunc_mapping))
# clean text
traintitle = traintitle.apply(lambda x: add_spaces(x))
traintitle = traintitle.apply(lambda x: clean_numbers(x))

In [52]:
traintitle.to_csv('traintitle.csv', header = False)

In [2]:
traintitle = pd.read_csv('traintitle.csv',header = None, index_col=[0])
traintilte = traintitle[1]

In [4]:
# build embedding vectors
embeds = np.zeros((traintitle.shape[0],300))
from nltk.tokenize import RegexpTokenizer 
tkz =  RegexpTokenizer('\s+', gaps = True)

In [5]:
for i in range(traintitle.shape[0]):
    for j in tkz.tokenize(traintitle.iloc[i][1]):
        try: 
            embeds[i] += embeddings_index.get(j)
        except:
            embeds[i] = embeds[i]
    embeds[i] = embeds[i]/len(tkz.tokenize(traintitle.iloc[i][1]))

In [12]:
# start building autoencoder model
import torch.nn as nn
from torch.autograd import Variable as V
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader

In [7]:
class AE0(nn.Module):
    def __init__(self, length):
        super().__init__()
        self.lin1 = nn.Linear(length,30)
#        self.lin1.bias.detach().zero_()
        self.lin2 = nn.Linear(30,length)
#        self.lin2.bias.detach().zero_()

    def forward(self, data):
        x = self.lin1(data)
        x = torch.tanh(x)
        x = self.lin2(x)
        return x

In [8]:
traint = torch.FloatTensor(embeds)
tloader = DataLoader(traint,batch_size=200)

In [9]:
model = AE0(embeds.shape[1])
loss=nn.MSELoss()
learning_rate = 1e-2
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
train_epochs = 10

In [13]:
for epoch in range(train_epochs):
    losses=[]
    dl = iter(tloader)
    for t in range(len(dl)):
        # Forward pass: compute predicted y and loss by passing x to the model.
        xt = next(dl)
        y_pred = model(V(xt))

        l = loss(y_pred,V(xt))
        losses.append(l)
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model parameters
        l.backward()

        # Calling the step function on an Optimizer makes an update to its parameters
        optimizer.step()

In [14]:
# find the the unique one
y_pred = model(V(traint))

In [16]:
y_pred = y_pred.detach().numpy()

In [17]:
diff = y_pred - embeds

In [31]:
diffs = np.square(diff)
error = np.sum(diffs, axis=1)

In [43]:
error = pd.DataFrame(error)

In [68]:
toshow = error.sort_values(by=[0],ascending = False)[0:10].index.values

In [78]:
traintitle.iloc[toshow,:]

,1
0,
626808,Dilbert dilbert
302351,Vine vine
1579457,Internships
728347,Join
1497519,Spam
1909421,Evden Eve Nakliyat sametnakliyat
834658,Quantitative dreamwidth
701692,Runtastic joins Adidas runtastic
691326,Nokia Nokia express
